## Berka Bank Data Preparation - Part 2 in SQL 


* This notebook demonstrates the versatility of SQL for Data Preprocessing and Data Cleaning.
* The dataset was loaded into the Database Management System: MS SQL Server v18.8

### Purpose: 
* to understanding datase and get it ready for analysis.
* to understand the data structure and identify issues with the dataset.
* to analyze with descriptive statistics.
* to handle missing values.
* to correct data type mismatch, including date data types.
* to standardize text data.
* to handle duplicate and inconsistent data.
* to validate the data.
* to add new tables and columns based on existing data for analysis purposes.

### Data Import & Relationships

1. Import all 8 csv dataset into MS SQL Server.
2. Set primary and foreign keys.
3. Set relationships.

* Understand the dataset: data, structure, content, and any potential issues with the dataset.
* View a sample of data from each table.

### Action: 
* Rename tables with conflicting names to SQL keywords: 
     * Rename the Order table to BankOrder
     * Rename the Transaction table to BankTransaction

In [ ]:
EXEC sp_rename 'Order', BankOrder;
EXEC sp_rename 'Transaction', BankTransaction;

In [2]:
import pandas as pd
import pyodbc
import warnings
warnings.filterwarnings('ignore')

### Connect to the BankDB Database

In [3]:
server = 'JAK-PC\\SQLEXPRESS'
database = 'BankDB'
driver = '{ODBC Driver 18 for SQL Server}'

conn_string = f'DRIVER={driver};SERVER={server};DATABASE={database};\
              Trusted_Connection=yes;Encrypt=no;TrustServerCertificate=yes'

try:
    conn = pyodbc.connect(conn_string)
    cursor = conn.cursor()
except pyodbc.Error as ex:
    print("Connection error:", ex)

#### Review tables to have better understanding of the dataset

In [ ]:
SELECT TOP(2) * FROM Account;
SELECT TOP(2) * FROM Client;
SELECT TOP(2) * FROM CreditCard;
SELECT TOP(2) * FROM Disposition;
SELECT TOP(2) * FROM District;
SELECT TOP(2) * FROM Loan;
SELECT TOP(2) * FROM BankOrder;
SELECT TOP(2) * FROM BankTransaction;

### Action: Transform dates into standard form on 4 tables: 

<blockquote>
Convert the IssueDate field on the CreditCard table to Date.
In SQL:    
    
```UPDATE CreditCard SET IssuedDate = CONVERT(Date, IssuedDate);```
</blockquote>

<blockquote>
Convert the EntryDate field on the Account table to Date.
In SQL: 
    
```UPDATE Account SET EntryDate = CONVERT(Date, EntryDate);```
</blockquote>

<blockquote>
Convert the EntryDate field on the BankTransaction table to Date.
In SQL:
    
```UPDATE Loan SET EntryDate = CONVERT(Date, EntryDate);```
</blockquote>

<blockquote>
Convert the EntryDate field on the Loan table to Date.
In SQL:
    
```UPDATE BankTransaction SET EntryDate = CONVERT(Date, EntryDate);```
</blockquote>    

In [ ]:
UPDATE CreditCard SET IssuedDate = CONVERT(Date, IssuedDate);
UPDATE Account SET EntryDate = CONVERT(Date, EntryDate);
UPDATE Loan SET EntryDate = CONVERT(Date, EntryDate);
UPDATE BankTransaction SET EntryDate = CONVERT(Date, EntryDate);

### View Table Structures and Schema

#### Purpose:
* To see column names, data types, and constraints. 
* To correct and convert data to appropriate types.

In [ ]:
EXEC sp_help CreditCard;
EXEC sp_help Disposition;
EXEC sp_help Client;
EXEC sp_help District;
EXEC sp_help Account;
EXEC sp_help BankOrder;
EXEC sp_help BankTransaction;
EXEC sp_help Loan;

### Action: 
1. ```Client``` Table: Convert ```BirthNumber``` from string to integer
    
2. ```Account``` Table: Convert ```EntryDate``` to Date data type
    
3. ```CreditCard``` Table: Convert ```IssuedDate``` to Date data type.
    
4. ```Loan``` Table: 
	* Convert ```EntryDate``` to Date data type
	* Convert ```Amount``` from string to number
	* Convert ```Payments``` from string to number
	* Convert ```Duration``` from string to integer
5. ```BankOrder``` Table: Convert ```Amount``` from string to number

6. ```BankTransaction``` Table: 
	* Convert ```EntryDate``` to Date data type
	* Convert ```Amount``` from string to number
	* Convert ```Balance``` from string to number

In [ ]:
ALTER TABLE Client
ALTER COLUMN BirthNumber INT;

ALTER TABLE Account
ALTER COLUMN EntryDate DATE;

ALTER TABLE CreditCard
ALTER COLUMN IssuedDate DATE;

ALTER TABLE Loan
ALTER COLUMN EntryDate DATE;

ALTER TABLE Loan
ALTER COLUMN Amount DECIMAL(10,2);

ALTER TABLE Loan
ALTER COLUMN Payments DECIMAL(10,2);

ALTER TABLE Loan
ALTER COLUMN Duration INT;

ALTER TABLE BankOrder
ALTER COLUMN Amount DECIMAL(10,2);

ALTER TABLE BankTransaction
ALTER COLUMN EntryDate DATE;

ALTER TABLE BankTransaction
ALTER COLUMN Amount DECIMAL(10,2);

ALTER TABLE BankTransaction
ALTER COLUMN Balance DECIMAL(10,2);

### Resolve Missing Values

* Identify missing values: 
    * Determine if values are missing in a specific column, which can affect data quality and analysis.
* Use ```SELECT COUNT(*), COUNT(column_name)```: the query helps to understand the dataset, the data structure, and potential issues.

In [4]:
def fetch_data(query_string):
    """
    fetch_function consumes query_string (a SQL Query Statement), and 
    produces df, a pandas dataframe that contains the result of the SQL query.
    """
    df = pd.DataFrame()

    try:
        df = pd.read_sql_query(query_string, conn)
    except pyodbc.Error as ex:
        print("Connection error:", ex) 

    blank_row_index = [''] * len(df)
    df.index = blank_row_index
    
    return df

In [5]:
query_string = """
    SELECT 
        COUNT(*) AS CountRows,
        COUNT(CardID) AS CountCardIDs,
        COUNT(Type) AS CountTypes,
        COUNT(IssuedDate) AS CountIssuedDates
    FROM 
        CreditCard
"""

df = fetch_data(query_string)
print("Credit Card Table")
df.head()    

Credit Card Table


,CountRows,CountCardIDs,CountTypes,CountIssuedDates
,892,892,892,892


In [6]:
query_string = """
    SELECT 
        COUNT(*) AS CountRows,
        COUNT(ClientID) AS CountClientIDs,
        COUNT(BirthNumber) AS CountBirthNumbers,
        COUNT(DistrictID) AS CountDistrictIDs
    FROM 
        Client
"""

df = fetch_data(query_string)
print("Client Table")
df.head()

Client Table


,CountRows,CountClientIDs,CountBirthNumbers,CountDistrictIDs
,5369,5369,5369,5369


In [7]:
query_string = """
    SELECT 
        COUNT(*) AS CountRows,
        COUNT(OrderID) AS CountOrderIDs,
        COUNT(AccountID) AS CountAccountIDs,
        COUNT(BankTo) AS CountBankTos,
        COUNT(AccountTo) AS CountAccountTos,
        COUNT(KSymbol) AS CountKSymbols,
        COUNT(Amount) AS CountAmounts
    FROM 
        BankOrder
"""

df = fetch_data(query_string)
print("Bank Order Table")
df.head()

Bank Order Table


,CountRows,CountOrderIDs,CountAccountIDs,CountBankTos,CountAccountTos,CountKSymbols,CountAmounts
,6471,6471,6471,6471,6471,6471,6471


In [8]:
query_string = """
    SELECT 
        COUNT(*) AS CountRows,
        COUNT(LoanID) AS CountLoanIDs,
        COUNT(AccountID) AS CountAccountIDs,
        COUNT(EntryDate) AS CountEntryDates,
        COUNT(Amount) AS CountAmounts,
        COUNT(Duration) AS CountDurations,
        COUNT(Payments) AS CountPayments,
        COUNT(StatusID) AS CountStatuseIDs
    FROM 
        Loan
"""

df = fetch_data(query_string)
print("Loan Table")
df.head()

Loan Table


,CountRows,CountLoanIDs,CountAccountIDs,CountEntryDates,CountAmounts,CountDurations,CountPayments,CountStatuseIDs
,682,682,682,682,682,682,682,682


In [9]:
query_string = """
    SELECT 
        COUNT(*) AS CountRows,
        COUNT(DispositionID) AS CountDispositionIDs,
        COUNT(ClientID) AS CountClientIDs,
        COUNT(Type) AS CountTypes
    FROM 
        Disposition
"""

df = fetch_data(query_string)
print("Disposition Table")
df.head()

Disposition Table


,CountRows,CountDispositionIDs,CountClientIDs,CountTypes
,5369,5369,5369,5369


In [10]:
query_string = """
    SELECT 
        COUNT(*) AS CountRows,
        COUNT(AccountID) AS CountAccountIDs,
        COUNT(DistrictID) AS CountDistrictIDs,
        COUNT(Frequency) AS CountFrequencys,
        COUNT(EntryDate) AS CountEntryDates
    FROM 
        Account
"""

df = fetch_data(query_string)
print("Account Table")
df.head()

Account Table


,CountRows,CountAccountIDs,CountDistrictIDs,CountFrequencys,CountEntryDates
,4500,4500,4500,4500,4500


In [11]:
query_string = """
    SELECT 
        COUNT(*) AS CountRows,
        COUNT(TransactionID) AS CountTransactionIDs,
        COUNT(AccountID) AS CountAccountIDs,
        COUNT(EntryDate) AS CountEntryDates,
        COUNT(Type) AS CountTypes,
        COUNT(OperationID) AS CountOperationIDs,
        COUNT(Amount) AS CountAmounts,
        COUNT(Balance) AS CountBalances,
        COUNT(KSymbol) AS CountKSymbols,
        COUNT(Bank) AS CountBanks,
        COUNT(Account) AS CountAccounts
    FROM 
        BankTransaction
"""

df = fetch_data(query_string)
print("BankTransaction Table")
df.head()

BankTransaction Table


,CountRows,CountTransactionIDs,CountAccountIDs,CountEntryDates,CountTypes,CountOperationIDs,CountAmounts,CountBalances,CountKSymbols,CountBanks,CountAccounts
,1056320,1056320,1056320,1056320,1056320,873206,1056320,1056320,1056320,1056320,1056320


Note that OperationsIDs in the "BankTransaction Table are missing some values: 873,206 entries out of a possible 1,056,320.

In [12]:
query_string = """
    SELECT 
        COUNT(*) AS CountRows,
        COUNT(DistrictID) AS CountDistrictIDs,
        COUNT(A2) AS CountA2Values,
        COUNT(A3) AS CountA3Values,
        COUNT(A4) AS CountA4Values,
        COUNT(A5) AS CountA5Values,
        COUNT(A6) AS CountA6Values,
        COUNT(A7) AS CountA7Values,
        COUNT(A8) AS CountA8Values,
        COUNT(A9) AS CountA9Values,
        COUNT(A10) AS CountA10Values,
        COUNT(A11) AS CountA11Values,
        COUNT(A12) AS CountA12Values,
        COUNT(A13) AS CountA13Values,
        COUNT(A14) AS CountA14Values,
        COUNT(A15) AS CountA15Values,
        COUNT(A16) AS CountA16Values
    FROM 
        District
"""

df = fetch_data(query_string)
print("District Table")
df.head()

District Table


,CountRows,CountDistrictIDs,CountA2Values,CountA3Values,CountA4Values,CountA5Values,CountA6Values,CountA7Values,CountA8Values,CountA9Values,CountA10Values,CountA11Values,CountA12Values,CountA13Values,CountA14Values,CountA15Values,CountA16Values
,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77


### Resolve Duplicate Values.

* Check for the **uniqueness of primary keys to avoid duplicated primary key values**.
* Check for uniqueness: Verify if each value in a column is unique, 
    ensuring **data integrity** and preventing potential issues in operations or analysis that rely on unique identifiers.
* This should be done with only primary key values.
* Use ```SELECT COUNT(*), COUNT(DISTINCT column_name)``` 

In [ ]:
SELECT 
    COUNT(*) AS CountRows,
    COUNT(DISTINCT CardID) AS CountUniqueCardIDs
FROM 
    CreditCard;

SELECT 
    COUNT(*) AS CountRows,
    COUNT(DISTINCT DispositionID) AS CountUniqueDispositionIDs
FROM 
    Disposition;

SELECT 
    COUNT(*) AS CountRows,
    COUNT(DISTINCT ClientID) AS CountUniqueClientIDs
FROM 
    Client;

SELECT 
    COUNT(*) AS CountRows,
    COUNT(DISTINCT OrderID) AS CountUniqueOrderIDs
FROM 
    BankOrder;

SELECT 
    COUNT(*) AS CountRows,
    COUNT(DISTINCT TransactionID) AS CountUniqueTransactionIDs
FROM 
    BankTransaction;

SELECT 
    COUNT(*) AS CountRows,
    COUNT(DISTINCT LoanID) AS CountUniqueLoanIDs
FROM 
    Loan;

SELECT 
    COUNT(*) AS CountRows,
    COUNT(DISTINCT DistrictID) AS CountUniqueDistrictIDs
FROM 
    District;

SELECT 
    COUNT(*) AS CountRows,
    COUNT(DISTINCT AccountID) AS CountUniqueAccountIDs
FROM 
    Account;

### Descriptive Stats: 

* Use descriptive statistics: MIN, MAX, SUM, COUNT, AVG to analyze the data.

In [ ]:
SELECT 
    MIN(CardID) AS MinCardID,
    MAX(CardID) AS MaxCardID,
    MIN(Type) AS MinType,
    MAX(Type) AS MaxType, 
    MIN(IssuedDate) AS MinIssuedDate,
    MAX(IssuedDate) AS MaxIssuedDate
FROM 
    CreditCard;

SELECT 
    MIN(DispositionID) AS MinDispositionID,
    MAX(DispositionID) AS MaxDispositionID,
    MIN(ClientID) AS MinClientID,
    MAX(ClientID) AS MaxClientID,
    MIN(Type) AS MinType,
    MAX(Type) AS MaxType
FROM 
    Disposition;

SELECT 
    MIN(ClientID) AS MinClientID,
    MAX(ClientID) AS MaxClientID,
    MIN(BirthNumber) AS MinBirthNumber,
    MAX(BirthNumber) AS MaxBirthNumber,
    MIN(DistrictID) AS MinDistrictID,
    MAX(DistrictID) AS MaxDistrictID
FROM 
    Client;

SELECT 
    MIN(OrderID) AS MinOrderID,
    MAX(OrderID) AS MaxOrderID,
    MIN(AccountID) AS MinAccountID,
    MAX(AccountID) AS MaxAccountID,
    MIN(BankTo) AS MinBankTo,
    MAX(BankTo) AS MaxBankTo,
    MIN(AccountTo) AS MinAccountTo,
    MAX(AccountTo) AS MaxAccountTo,
    MIN(KSymbol) AS MinKSymbol,
    MAX(KSymbol) AS MaxKSymbol,
    MIN(Amount) AS MinAmount,
    MAX(Amount) AS MaxAmount
FROM 
    BankOrder;

SELECT 
    MIN(TransactionID) AS MinTransactionID,
    MAX(TransactionID) AS MaxTransactionID,
    MIN(AccountID) AS MinAccountID,
    MAX(AccountID) AS MaxAccountID,
    MIN(EntryDate) AS MinEntryDate,
    MAX(EntryDate) AS MaxEntryDate,
    MIN(Type) AS MinType,
    MAX(Type) AS MaxType,
    MIN(Operation) AS MinOperation,
    MAX(Operation) AS MaxOperation,
    MIN(Amount) AS MinAmount,
    MAX(Amount) AS MaxAmount,
    MIN(Balance) AS MinBalance,
    MAX(Balance) AS MaxBalance,
    MIN(KSymbol) AS MinKSymbol,
    MAX(KSymbol) AS MaxKSymbol,
    MIN(Bank) AS MinBank,
    MAX(Bank) AS MaxBank,
    MIN(Account) AS MinAccount,
    MAX(Account) AS MaxAccount
FROM 
    BankTransaction;

SELECT 
    MIN(LoanID) AS MinLoanID,
    MAX(LoanID) AS MaxLoanID,
    MIN(AccountID) AS MinAccountID,
    MAX(AccountID) AS MaxAccountID,
    MIN(EntryDate) AS MinEntryDate,
    MAX(EntryDate) AS MaxEntryDate,
    MIN(Amount) AS MinAmount,
    MAX(Amount) AS MaxAmount,
    MIN(Duration) AS MinDuration,
    MAX(Duration) AS MaxDuration,
    MIN(Payments) AS MinPayments,
    MAX(Payments) AS MaxPayments,
    MIN(Status) AS MinStatus,
    MAX(Status) AS MaxStatus
FROM 
    Loan;

SELECT 
    MIN(DistrictID) AS MinDistrictID,
    MAX(DistrictID) AS MaxDistrictID,
    MIN(A2) AS MinA2,
    MAX(A2) AS MaxA2,
    MIN(A3) AS MinA3,
    MAX(A3) AS MaxA3,
    MIN(A4) AS MinA4,
    MAX(A4) AS MaxA4,
    MIN(A5) AS MinA5,
    MAX(A5) AS MaxA5,
    MIN(A6) AS MinA6,
    MAX(A6) AS MaxA6,
    MIN(A7) AS MinA7,
    MAX(A7) AS MaxA7,
    MIN(A8) AS MinA8,
    MAX(A8) AS MaxA8,
    MIN(A9) AS MinA9,
    MAX(A9) AS MaxA9,
    MIN(A10) AS MinA10,
    MAX(A10) AS MaxA10,
    MIN(A11) AS MinA11,
    MAX(A11) AS MaxA11,
    MIN(A12) AS MinA12,
    MAX(A12) AS MaxA12,
    MIN(A13) AS MinA13,
    MAX(A13) AS MaxA13,
    MIN(A14) AS MinA14,
    MAX(A14) AS MaxA14,
    MIN(A15) AS MinA15,
    MAX(A15) AS MaxA15,
    MIN(A16) AS MinA16,
    MAX(A16) AS MaxA16
FROM 
    District;

SELECT 
    MIN(AccountID) AS MinAccountID,
    MAX(AccountID) AS MaxAccountID,
    MIN(DistrictID) AS MinDistrictID,
    MAX(DistrictID) AS MaxDistrictID,
    MIN(Frequency) AS MinFrequency,
    MAX(Frequency) AS MaxFrequency,
    MIN(EntryDate) AS MinEntryDate,
    MAX(EntryDate) AS MaxEntryDate
FROM 
    Account;

**Few observations from the data:**

* `Account` Table: 
    * MIN EntryDate = 1993-01-01
    * MAX EntryDate = 1997-12-29
* `Loan` Table: 
    * Min Status: A
    * Max Status: D
* `BankTransaction` Table:
    * Min EntryDate = 1993-01-01
    * Max EntryDate = 1998-12-31
* `District` Table:
    * Min DistrictID is 1
    * Max DistrictID is 9

### BankTransaction Table

`BankTransation` table has an `OperationID` field:

	* It provides a description of the type of transation done
	* But, the field's values are difficult to easily process. See the values below.
        * VYBER KARTOU => Credit Card Withdrawal
        * VKLAD => Cash Deposit
        * PREVOD Z UCTU => Deposit from another bank
        * VYBER => Cash Withdrawal
        * PREVOD NA UCET => Remittance to another bank

**Action**: Create a new table that will capture this information and use shorter strings to represent the values as follows.

	* WCC: VYBER KARTOU => Credit Card Withdrawal
	* DC: VKLAD => Cash Deposit
	* DB: PREVOD Z UCTU => Deposit from another bank
	* WC: VYBER => Cash Withdrawal
	* WB: PREVOD NA UCET => Remittance to another bank
    

**Create TransactionOperation table**
```
CREATE TABLE TransactionOperation (
    OperationID VARCHAR(3) PRIMARY KEY,
    Description VARCHAR(50),
    OriginalCode VARCHAR(50)
);
```


**Insert data into TransactionOperation**
```
INSERT INTO TransactionOperation (OperationID, Description, OriginalCode)
VALUES
    ('WCC', 'Credit Card Withdrawal', 'VYBER KARTOU'),
    ('DC', 'Cash Deposit', 'VKLAD'),
    ('DB', 'Deposit from another bank', 'PREVOD Z UCTU'),
    ('WC', 'Cash Withdrawal', 'VYBER'),
    ('WB', 'Remittance to another bank', 'PREVOD NA UCET');
```

**Add new column BankOperation to BankTransaction table**
```
ALTER TABLE BankTransaction
ADD OperationID VARCHAR(3);
```

**Alter BankTransaction table to add foreign key constraint**
```
ALTER TABLE BankTransaction
ADD CONSTRAINT FK_BankTransaction_Operation
FOREIGN KEY (OperationID) REFERENCES TransactionOperation(OperationID);
```

**Populate BankTransaction.OperationID based on TransactionOperation.OriginalCode**
```
UPDATE BankTransaction
SET OperationID = TranOp.OperationID
FROM BankTransaction BT
JOIN TransactionOperation TranOp ON BT.Operation = TranOp.OriginalCode;
```

**Drop the Operation column on BankTransaction table**
```
ALTER TABLE BankTransaction
DROP COLUMN Operation;
```

**Update values in BankTransaction table for easier comprehension**
```
UPDATE BankTransaction
SET Type = 'Withdraw'
WHERE Type = 'VYDAJ'

UPDATE BankTransaction
SET Type = 'Deposit'
WHERE Type = 'PRIJEM'
```

#### Loan Operations 

* Loan status represents the status of clients paying off their loans.
* There are 4 states: `A, B, C, D`.
* Create `LoanStatus` table as follows.

        LoanStatus:	
        StatusID (PK)	Description
        A	Contract finished, no problems
        B	Contract finished, loan not payed
        C	Running contract, OK thus-far
        D	Running contract, client in debt
* There's going to be a PK-FK relationship with the Status column in the `Loan` table.



**Create LoanStatus table**
```
CREATE TABLE LoanStatus (
    StatusID VARCHAR(1) PRIMARY KEY,
    Description VARCHAR(50)
);
```

**Insert data into LoanStatus**
```
INSERT INTO LoanStatus (StatusID, Description)
VALUES
    ('A', 'Contract finished, no problems'),
    ('B', 'Contract finished, loan not paid'),
    ('C', 'Running contract, OK thus-far'),
    ('D', 'Running contract, client in debt');
```

**Add new column StatusID to Loan table**
```ALTER TABLE Loan
ADD StatusID VARCHAR(1);
```

**Create PK-FK relationship between LoanStatus and Loan**
```ALTER TABLE Loan
ADD CONSTRAINT FK_Loan_Status
FOREIGN KEY (StatusID) REFERENCES LoanStatus(StatusID);
```

**Update Loan.StatusID based on LoanStatus.StatusID**
```UPDATE Loan
SET StatusID = LS.StatusID
FROM Loan L
JOIN LoanStatus LS ON L.Status = LS.StatusID;
```

**Drop the Status column from the Loan table**
```ALTER TABLE Loan
DROP COLUMN Status;
```

In [13]:
try:
    cursor.close()
    conn.close()
except pyodbc.Error as ex:
    print("Connection error:", ex)

### End of Data Preparation - Part 2

* Next step is to create another Jupyter Notebook for Exploratory Data Analysis, mostly to discover meaningful information and insights from the dataset.
* The objective is to generate data that will be uploaded into Power BI, Tableau, Excel, or Google Sheet to create visualizations and derive insights.